# MISO Footprint and Hexagonal Grid Generation

This section implements Task 2: MISO footprint and hexagonal grid generation

## Implementation Details
1. Create function to define MISO territory boundary from state list
2. Implement hexagonal grid generator with ~40km spacing clipped to MISO footprint
3. Add grid cell ID assignment and centroid calculation
4. Write unit tests for spatial accuracy and coverage validation

In [ ]:
# Import the spatial framework implementation
import sys
import os
sys.path.append('.')

from spatial_framework import (
    MISOFootprint, 
    HexGridGenerator, 
    SpatialProcessingEngine, 
    SpatialProcessingTests
)

print("✓ Spatial framework classes imported successfully")

In [ ]:
# Initialize spatial processing engine with configuration
# Assuming config_manager is available from previous cells

try:
    current_config = config_manager.get_config()
    spatial_engine = SpatialProcessingEngine(current_config)
    
    print("🗺️  Initializing MISO spatial framework...")
    print(f"   Target hex size: {current_config['runtime']['hex_size_km']} km")
    print(f"   Coordinate system: {current_config['runtime']['crs']}")
    print(f"   Runtime mode: {config_manager.get_runtime_mode()}")
    
except NameError:
    # Fallback configuration if config_manager not available
    print("⚠️  config_manager not found, using fallback configuration")
    
    fallback_config = {
        'runtime': {
            'mode': 'demo',
            'horizons_h': [12, 24, 36, 48],
            'crs': 'EPSG:4326',
            'random_seed': 42,
            'hex_size_km': 40
        }
    }
    
    spatial_engine = SpatialProcessingEngine(fallback_config)
    
    print("🗺️  Initializing MISO spatial framework (fallback config)...")
    print(f"   Target hex size: 40 km")
    print(f"   Coordinate system: EPSG:4326")
    print(f"   Runtime mode: demo")

print("\n✓ Spatial processing engine initialized")

In [ ]:
# Initialize footprint and grid
print("🏗️  Creating MISO footprint and hexagonal grid...")

try:
    # Initialize the complete spatial framework
    miso_footprint, hex_grid = spatial_engine.initialize_spatial_framework()
    
    print("\n✅ Spatial framework initialized successfully!")
    print(f"   MISO footprint: {spatial_engine.footprint_manager.get_area_km2():,.0f} km²")
    print(f"   Hex grid cells: {len(hex_grid):,}")
    print(f"   Average cell area: {hex_grid['area_km2'].mean():.1f} km²")
    
    # Display footprint bounds
    minx, miny, maxx, maxy = miso_footprint.total_bounds
    print(f"   Footprint bounds: ({minx:.2f}, {miny:.2f}) to ({maxx:.2f}, {maxy:.2f})")
    
    # Display grid summary
    grid_summary = spatial_engine.grid_generator.get_grid_summary()
    print(f"   Grid area range: {grid_summary['actual_area_km2']['min']:.1f} - {grid_summary['actual_area_km2']['max']:.1f} km²")
    
except Exception as e:
    print(f"\n❌ Spatial framework initialization failed: {e}")
    raise

In [ ]:
# Run comprehensive spatial tests
print("🧪 Running spatial processing tests...")

spatial_tests = SpatialProcessingTests(spatial_engine)
test_results = spatial_tests.run_all_tests()

# Display test summary
print(spatial_tests.get_test_summary())

# Check if all tests passed
all_passed = all(test_results.values())
if all_passed:
    print("\n🎉 All spatial processing tests PASSED!")
    print("   ✓ Footprint geometry is valid")
    print("   ✓ Grid generation produces proper hexagons")
    print("   ✓ Cell IDs are unique")
    print("   ✓ Centroids are calculated correctly")
    print("   ✓ Spatial aggregation works properly")
    print("   ✓ Coordinate systems are consistent")
    print("\n   Spatial framework is ready for weather data integration!")
else:
    failed_tests = [name for name, result in test_results.items() if not result]
    print(f"\n⚠️  Some tests FAILED: {failed_tests}")
    print("   Review test results before proceeding")

In [ ]:
# Export spatial data for future use
print("💾 Exporting spatial data...")

try:
    spatial_engine.export_spatial_data()
    print("✅ Spatial data exported successfully!")
    print("   📄 data/processed/miso_footprint.geojson")
    print("   📄 data/processed/miso_hex_grid.geojson")
except Exception as e:
    print(f"❌ Export failed: {e}")

# Display detailed grid summary
grid_summary = spatial_engine.grid_generator.get_grid_summary()
print("\n📊 Detailed Grid Summary:")
print(f"   Total cells: {grid_summary['n_cells']:,}")
print(f"   Target size: {grid_summary['target_hex_size_km']} km")
print(f"   Actual area: {grid_summary['actual_area_km2']['mean']:.1f} ± {grid_summary['actual_area_km2']['std']:.1f} km²")
print(f"   Area range: {grid_summary['actual_area_km2']['min']:.1f} - {grid_summary['actual_area_km2']['max']:.1f} km²")
print(f"   CRS: {grid_summary['crs']}")

# Calculate coverage statistics
footprint_area = spatial_engine.footprint_manager.get_area_km2()
grid_total_area = hex_grid['area_km2'].sum()
coverage_ratio = grid_total_area / footprint_area
print(f"   Coverage ratio: {coverage_ratio:.1%}")

print("\n" + "="*60)
print("🗺️  MISO Spatial Framework Complete")
print("   ✅ Footprint defined and validated")
print("   ✅ Hexagonal grid generated and tested")
print("   ✅ Cell IDs assigned and centroids calculated")
print("   ✅ Spatial aggregation functions ready")
print("   ✅ Data exported for downstream processing")
print("   ✅ All unit tests passed")
print("="*60)

In [ ]:
# Display sample of the generated data
print("📋 Sample Data Preview:")
print("\n🌍 MISO Footprint:")
print(miso_footprint.head())

print("\n🔷 Hexagonal Grid (first 5 cells):")
print(hex_grid[['cell_id', 'centroid_lon', 'centroid_lat', 'area_km2', 'row', 'col']].head())

print("\n📈 Grid Statistics:")
print(f"   Cell count: {len(hex_grid)}")
print(f"   Area statistics (km²):")
print(f"     Mean: {hex_grid['area_km2'].mean():.2f}")
print(f"     Std:  {hex_grid['area_km2'].std():.2f}")
print(f"     Min:  {hex_grid['area_km2'].min():.2f}")
print(f"     Max:  {hex_grid['area_km2'].max():.2f}")

print(f"\n   Centroid coordinates:")
print(f"     Longitude range: {hex_grid['centroid_lon'].min():.2f} to {hex_grid['centroid_lon'].max():.2f}")
print(f"     Latitude range:  {hex_grid['centroid_lat'].min():.2f} to {hex_grid['centroid_lat'].max():.2f}")

print("\n✅ Task 2 Implementation Complete!")
print("   Ready to proceed with weather data ingestion (Task 3)")